In [1]:
#standard imports
import pandas as pd
import numpy as np
from io import BytesIO
#specialized imports
import re
import io
import os
import stockfish

In [2]:
#importing the raw data (courtesy of Maven Analytics)
games_df = pd.read_csv(r"C:\Users\CDHan\Documents\Online+Chess+Games\chess_games.csv")
#checking the dataframe
print(games_df)

In [8]:
#checking ratings spread.
print("Highest White rating" + str(games_df['white_rating'].max()))
print("Highest Black rating" + str(games_df['black_rating'].max()))

Highest White rating2700
Highest Black rating2723


In [10]:
#defining a function to determine rating brackets
def ratingbracket(x):
    if x >= 2500:
        return "Grandmaster"
    elif x >= 2400:
        return "International Master"
    elif x >= 2300:
        return "FIDE Master"
    elif x >= 2000:
        return "Candidate Master"
    elif x >= 1800:
        return "Class A"
    elif x >= 1600:
        return "Class B"
    elif x >= 1400:
        return "Class C"
    elif x >= 1200:
        return "Class D"
    elif x >= 1000:
        return "Class E"
    elif x < 1000:
        return "Class D"
    else:
        return "Error"
#Assumes Lichess rating maps to FIDE ELO (counterfactual)

In [12]:
#bracketting player ratings 
games_df['black_bracket'] = games_df['black_rating'].apply(ratingbracket)
games_df['white_bracket'] = games_df['white_rating'].apply(ratingbracket)

In [17]:
#defining the function to split time and increment
def extract_time_and_increment(data):
    time, increment = data.split("+")
    return str(time), str(increment)

In [19]:
#checking the time and increment values for formatting issues
print(games_df['time_increment'].unique())

['15+2' '5+10' '20+0' '30+3' '10+0' '15+30' '15+0' '20+60' '5+40' '8+0'
 '15+15' '15+16' '11+0' '30+60' '15+5' '10+10' '15+3' '18+0' '30+30'
 '20+25' '5+5' '30+0' '40+40' '16+0' '6+10' '8+10' '8+12' '7+12' '15+10'
 '20+5' '20+10' '45+0' '5+8' '30+2' '25+0' '10+2' '7+2' '25+25' '7+8'
 '6+6' '10+4' '12+0' '15+1' '2+10' '9+0' '10+9' '6+7' '30+10' '30+15'
 '10+3' '45+30' '9+2' '7+3' '6+5' '25+5' '5+7' '8+2' '20+8' '30+5' '25+15'
 '15+7' '8+8' '5+9' '16+7' '10+7' '7+5' '40+20' '10+5' '11+5' '10+15'
 '180+0' '17+5' '8+5' '3+8' '60+120' '90+120' '15+8' '9+1' '9+3' '2+15'
 '10+1' '5+12' '60+10' '5+60' '8+15' '30+25' '12+1' '14+0' '60+20' '10+8'
 '20+3' '14+12' '7+7' '60+0' '45+45' '4+15' '5+6' '180+180' '8+3' '4+6'
 '10+25' '3+10' '7+4' '6+3' '5+15' '45+15' '15+20' '6+8' '6+9' '14+8'
 '6+4' '15+4' '9+20' '20+15' '20+2' '9+10' '8+7' '13+1' '150+180' '3+12'
 '7+10' '10+60' '5+30' '20+30' '14+10' '25+10' '8+1' '19+0' '9+12' '20+1'
 '0+25' '0+15' '15+6' '8+6' '10+6' '25+8' '15+14' '20+20' '13+10' 

In [23]:
#splitting time and increment into their own columns
time_increment = games_df['time_increment'].str.split('+', expand=True)
games_df['Time'] = time_increment[0].astype(int)
games_df['Increment'] = time_increment[1].astype(int)

In [25]:
#defining the function to interpret the time control
def timecontrol(minutes,increment):
    if int(minutes)+int(increment) >= 30:
        return "Classical"
    elif int(minutes)+int(increment) >= 10:
        return "Rapid"
    elif int(minutes)+int(increment) >= 3:
        return "Blitz"
    elif int(minutes)+int(increment) >= 1:
        return "Bullet"
    else:
        return "Hyperbullet"

In [29]:
#applying the time control function
games_df['TimeControlCode'] = games_df.apply(lambda row: timecontrol(row['Time'], row['Increment']), axis=1)

In [35]:
#copying columns to a new dataframe where I will chunk-analyze with stockfish.
games2_df = games_df[["game_id","moves"]].copy()

In [36]:
# Define the column names
columns = []
for i in range(1, 11):
    columns.extend([f"WhiteMove{i}", f"WhiteEvaluation{i}", f"WhiteBestMove{i}", f"BlackMove{i}", f"BlackEvaluation{i}", f"BlackBestMove{i}"])
# Add empty columns to the existing DataFrame
for column in columns:
    games2_df[column] = None
print(games2_df.columns)


Index(['game_id', 'moves', 'WhiteMove1', 'WhiteEvaluation1', 'WhiteBestMove1',
       'BlackMove1', 'BlackEvaluation1', 'BlackBestMove1', 'WhiteMove2',
       'WhiteEvaluation2', 'WhiteBestMove2', 'BlackMove2', 'BlackEvaluation2',
       'BlackBestMove2', 'WhiteMove3', 'WhiteEvaluation3', 'WhiteBestMove3',
       'BlackMove3', 'BlackEvaluation3', 'BlackBestMove3', 'WhiteMove4',
       'WhiteEvaluation4', 'WhiteBestMove4', 'BlackMove4', 'BlackEvaluation4',
       'BlackBestMove4', 'WhiteMove5', 'WhiteEvaluation5', 'WhiteBestMove5',
       'BlackMove5', 'BlackEvaluation5', 'BlackBestMove5', 'WhiteMove6',
       'WhiteEvaluation6', 'WhiteBestMove6', 'BlackMove6', 'BlackEvaluation6',
       'BlackBestMove6', 'WhiteMove7', 'WhiteEvaluation7', 'WhiteBestMove7',
       'BlackMove7', 'BlackEvaluation7', 'BlackBestMove7', 'WhiteMove8',
       'WhiteEvaluation8', 'WhiteBestMove8', 'BlackMove8', 'BlackEvaluation8',
       'BlackBestMove8', 'WhiteMove9', 'WhiteEvaluation9', 'WhiteBestMove9',
     

In [38]:
#Writing to CSV files for hard storage.
games2_df.to_csv(r"C:\Users\CDHan\Documents\Online+Chess+Games\chess_games_to_analyze.csv", index=False)
games_df.to_csv(r"C:\Users\CDHan\Documents\Online+Chess+Games\chess_games_processed.csv", index=False)